In [1]:
# %load ../firstcell.py
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [40]:
from functools import reduce

In [41]:
def deep_get(dictionary, keys, default=None):
    """Get the value from a dictionary using a list of keys.

    Args:
        dictionary (dict): The dictionary to retrieve the value from.
        keys (str): The keys to use, separated by periods.
        default: The default value to return if the key does not exist.

    Returns:
        The value from the dictionary at the specified keys, or the default value if the key does not exist.
    """
    return reduce(
        lambda d, key: d.get(key, default) if isinstance(d, dict) else default,
        keys.split("."),
        dictionary,
    )

In [129]:
class DotDict(dict):
    """A dictionary that allows access to its keys as if they were attributes.

    Args:
        value (dict): The dict object to access.

    """

    def __init__(self, value=None):
        if value is None:
            pass
        elif isinstance(value, dict):
            for key in value:
                self.__setitem__(key, value[key])
        else:
            raise TypeError("expected dict")

    def __getitem__(self, key):
        value = self.get(key, None)
        return value

    def __setitem__(self, key, value):
        if isinstance(value, dict) and not isinstance(value, DotDict):
            value = DotDict(value)
        if isinstance(value, list) and len(value) == 1 and isinstance(value[0], dict):
            value = DotDict(value[0])
        if (
            isinstance(value, list)
            and len(value) > 1
            and all(isinstance(v, dict) for v in value)
        ):
            value = DotDict({k: v for d in value for k, v in d.items()})
        super(DotDict, self).__setitem__(key, value)

    def __getstate__(self):
        return self.__dict__

    def __setstate__(self, d):
        self.__dict__.update(d)

    __setattr__, __getattr__ = __setitem__, __getitem__

In [130]:
def deep_set(dictionary, keys, value):
    """Set a value in a dictionary using a list of keys.

    Args:
        dictionary (dict): The dictionary to set the value in.
        keys (str): The keys to use, separated by periods.
        value: The value to set.
    """
    keys = keys.split(".")
    for key in keys[:-1]:
        if key in dictionary and not isinstance(dictionary[key], dict):
            raise TypeError(f"Expected dict type at key {key} in {dictionary}")

        if key not in dictionary:
            dictionary[key] = {}

        dictionary = dictionary[key]

    dictionary[keys[-1]] = value

In [131]:
car = {"brand": "Ford", "model": "Mustang", "year": 1964}

In [132]:
deep_set(car, "model", "Bronco")
print(car)

{'brand': 'Ford', 'model': 'Bronco', 'year': 1964}


In [133]:
# deep_set(car, "color", "red")
try:
    deep_set(car, "model.color", "blue")
except TypeError as e:
    print(e)

y = deep_get(car, "model")

print(y)
print(car)

Expected dict type at key model in {'brand': 'Ford', 'model': 'Bronco', 'year': 1964}
Bronco
{'brand': 'Ford', 'model': 'Bronco', 'year': 1964}


In [134]:
new_car = {
    "brand": "Ford",
    "model": {
        "name": "Mustang",
    },
    "year": 1964,
}

In [135]:
deep_set(new_car, "model.color", "blue")

y = deep_get(new_car, "model")

print(y)
print(new_car)

z = deep_get(new_car, "model.color")

print(z)

{'name': 'Mustang', 'color': 'blue'}
{'brand': 'Ford', 'model': {'name': 'Mustang', 'color': 'blue'}, 'year': 1964}
blue


In [138]:
def test_dot_dict():
    d = DotDict({"a": 1, "b": {"c": 2}})
    assert d.a == 1
    assert d.b.c == 2  # pyright: ignore[reportOptionalMemberAccess]
    d.b.c = 3  # pyright: ignore[reportOptionalMemberAccess]
    assert d.b.c == 3  # pyright: ignore[reportOptionalMemberAccess]


def test_deep_get():
    d = {"a": {"b": {"c": 1}}}
    assert deep_get(d, "a.b.c") == 1
    assert deep_get(d, "a.b.d") is None
    assert deep_get(d, "a.b.d", default=2) == 2


def test_deep_set():
    d = {}
    deep_set(d, "a.b.c", 1)
    assert d == {"a": {"b": {"c": 1}}}
    deep_set(d, "a.b.d", 2)
    assert d == {"a": {"b": {"c": 1, "d": 2}}}
    deep_set(d, "a.b", {"e": 3})
    assert d == {"a": {"b": {"e": 3}}}

In [139]:
test_dot_dict()
test_deep_get()
test_deep_set()